# pandas 실습1 

#### 제공된 3개의 파일을 활용한 실습
- user_ratings.dat 
- movies.dat 
- users.dat 

In [24]:
# 필요한 library import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

import warnings
warnings.filterwarnings(action = 'ignore') 

from datetime import date, datetime, timedelta
from dateutil.parser import parse

print("numpy version:",np.__version__) 
print("pandas version:",pd.__version__)

numpy version: 1.20.1
pandas version: 1.2.4


#### 1. 각 파일을 pd.read_csv() 로 읽어들인다
- 헤더 및 구분자 확인요망
- 3개 파일 모두 헤더X ,구분자는 :: 이다.

In [37]:
user_ratings =  pd.read_csv('C:/Users/whgud/data/user_ratings.dat',
                            sep = "::",
                            header= None,
                            names=['userId','movieId','rating','timestamp'])
movies       =  pd.read_csv('C:/Users/whgud/data/movies.dat',
                            sep = "::",
                            header= None,
                            names=['movieId','title','genres'])
users        =  pd.read_csv('C:/Users/whgud/data/users.dat',
                            sep = "::",
                            header= None)

In [38]:
# 데이터를 잘 불러왔는지 확인해보기
display(user_ratings.head())
display(movies.head())
display(users.head())

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [39]:
# users에 필요없는 열 제거
del users[3]
del users[4]
users.columns = ['userId','gender','age']
users

,userId,gender,age
0,1,F,1
1,2,M,56
2,3,M,25
3,4,M,45
4,5,M,25
...,...,...,...
6035,6036,F,25
6036,6037,F,45
6037,6038,F,56
6038,6039,F,45


In [40]:
# 3. 공통 칼럼을 이용한 데이터 병합하기

# movieid 칼럼을 이용해서 rating과 movie merge()함수로 병합
rating_movie = pd.merge(user_ratings,movies, on='movieId',how='left') # merge함수 사용해서 left join해줌.

# userid 칼럼을 이용해서 과 rating_movie과 user병합
data = pd.merge(rating_movie, users, on='userId',how='left') # merge함수 사용해서 left join해줌.
data  # 병합후 확인

,userId,movieId,rating,timestamp,title,genres,gender,age
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1
...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,M,25
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,M,25
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,M,25
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama,M,25


In [48]:
# 4. user 평점 개수 순위 확인(sort)
type(data)
ex_df = pd.DataFrame({
    'user_id_cnt' : data['userId'].value_counts()
})
ex_df.index.name = 'userId' # 행인덱스를 userid로 지정
ex_df = ex_df.reset_index() # 행인덱스를 reset해주면서 userid를 컬럼화
ex_df['user_id_cnt_rank'] = ex_df['user_id_cnt'].rank(ascending=False).astype(int) # 평점 개수 순위를 나타내는 칼럼을 새롭게 추가
display(ex_df)

# 기존 데이터에 ex_df 병합하기
data_result = pd.merge(data, ex_df, on = 'userId', how = 'left')
data_result

,userId,user_id_cnt,user_id_cnt_rank
0,4169,2314,1
1,1680,1850,2
2,4277,1743,3
3,1941,1595,4
4,1181,1521,5
...,...,...,...
6035,3787,20,5997
6036,4244,20,5997
6037,1452,20,5997
6038,421,20,5997


,userId,movieId,rating,timestamp,title,genres,gender,age,user_id_cnt,user_id_cnt_rank
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,53,4149
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,53,4149
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,53,4149
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,53,4149
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,53,4149
...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,M,25,341,778
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,M,25,341,778
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,M,25,341,778
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama,M,25,341,778


In [58]:
# 5.  각 영화의 성별 평균 평점
ex5_frm = data_result.loc[:,['movieId','gender','rating']]
ex5_frm
# groupby() 사용했을때
ex5_frm.groupby(['movieId','gender']).agg(np.mean) 


rating
movieId gender          
1       F       4.187817
        M       4.130552
2       F       3.278409
        M       3.175238
3       F       3.073529
...                  ...
3950    M       3.681818
3951    F       3.705882
        M       4.043478
3952    F       3.761905
        M       3.787986

[7152 rows x 1 columns]

In [113]:
# 5.  각 영화의 성별 평균 평점 & 평균평점(전체)
# pivot_table() 사용했을때
ex5_frm_result=ex5_frm.pivot_table(index='movieId',columns='gender',values = 'rating', aggfunc='mean',margins =True)
ex5_frm_result

gender,F,M,All
movieId,,,
1,4.187817,4.130552,4.146846
2,3.278409,3.175238,3.201141
3,3.073529,2.994152,3.016736
4,2.976471,2.482353,2.729412
5,3.212963,2.888298,3.006757
...,...,...,...
3949,3.950000,4.174107,4.115132
3950,3.600000,3.681818,3.666667
3951,3.705882,4.043478,3.900000


In [106]:
# 6.  평점이 220개 이상인 영화만 필터링
# groupby() 사용했을때
ex6_frm = data_result[['movieId','title','rating']]
ex6_frm_grp = ex6_frm.groupby(['movieId','title']).agg('count')  # groupby 결과를 새로운 변수로 받는다.
ex6_frm_grp.columns = ['rating_cnt'] # 칼럼 이름 알맞게 변경해주기
condition = ex6_frm_grp['rating_cnt'] >= 220 # 평점이 220개 이상인 조건 생성
ex6_frm_grp[condition] # 만들어준 조건으로 인덱싱

,,rating_cnt
movieId,title,
1,Toy Story (1995),2077
2,Jumanji (1995),701
3,Grumpier Old Men (1995),478
5,Father of the Bride Part II (1995),296
6,Heat (1995),940
...,...,...
3930,"Creature From the Black Lagoon, The (1954)",223
3932,"Invisible Man, The (1933)",232
3948,Meet the Parents (2000),862


In [122]:
# 7. 평점이 200개 이상인 영화들의 성별 평균 평점
# 평점이 200개 이상인 영화들만 추출.
# movieId 와 title로 groupby 한 ex6_frm_grp 데이터프레임에 조건만 변경하여 적용해서 추출
up_200 = ex6_frm_grp['rating_cnt'] >= 200
up_200_frm = ex6_frm_grp[up_200]
up_200_frm.info()
# 각 영화의 성별평점을 나타내는 ex5_frm_result과 up_200_frm 행인덱스 movieId를 기준으로 병합시켜주기
ex7_frm = up_200_frm.join(ex5_frm_result, how = 'left')
ex7_frm

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1426 entries, (1, 'Toy Story (1995)') to (3952, 'Contender, The (2000)')
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   rating_cnt  1426 non-null   int64
dtypes: int64(1)
memory usage: 203.8+ KB


,,rating_cnt,F,M,All
movieId,title,,,,
1,Toy Story (1995),2077,4.187817,4.130552,4.146846
2,Jumanji (1995),701,3.278409,3.175238,3.201141
3,Grumpier Old Men (1995),478,3.073529,2.994152,3.016736
5,Father of the Bride Part II (1995),296,3.212963,2.888298,3.006757
6,Heat (1995),940,3.682171,3.909988,3.878723
...,...,...,...,...,...
3930,"Creature From the Black Lagoon, The (1954)",223,3.121951,3.285714,3.255605
3932,"Invisible Man, The (1933)",232,3.681818,3.765957,3.750000
3948,Meet the Parents (2000),862,3.619048,3.641838,3.635731


In [131]:
# 8. 여성이 가장 좋아하는 영화 순위

rank = ex7_frm.sort_values(by="F", ascending =False)["F"].rank(ascending = False)# 여자의 평균평점을 나타내는 "F" 칼럼을 기준으로 내림차순 정렬 후 순위 부여
ex7_frm['female_rank'] = rank # female_rank열로 여성이 가장 좋아하는 영화 순위 추가 
rank_sorted_f =ex7_frm.sort_values(by="female_rank",ascending=True) # 순위별로 정렬한 것.
rank_sorted_f

,,rating_cnt,F,M,All,female_rank
movieId,title,,,,,
745,"Close Shave, A (1995)",657,4.644444,4.473795,4.520548,1.0
1148,"Wrong Trousers, The (1993)",882,4.588235,4.478261,4.507937,2.0
3022,"General, The (1927)",206,4.575758,4.329480,4.368932,3.0
922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),470,4.572650,4.464589,4.491489,4.0
720,Wallace & Gromit: The Best of Aardman Animation (1996),438,4.563107,4.385075,4.426941,5.0
...,...,...,...,...,...,...
2412,Rocky V (1990),274,1.878788,2.132780,2.102190,1422.0
1389,Jaws 3-D (1983),210,1.863636,1.851064,1.852381,1423.0
193,Showgirls (1995),217,1.709091,2.166667,2.050691,1424.0


In [134]:
# 9. 남녀간의 평균 평점 차이
ex7_frm['gender_diff']= (ex7_frm['F'] - ex7_frm['M']).abs() # 차이값을 절대값으로! 
ex7_frm

,,rating_cnt,F,M,All,female_rank,gender_diff
movieId,title,,,,,,
1,Toy Story (1995),2077,4.187817,4.130552,4.146846,112.0,0.057265
2,Jumanji (1995),701,3.278409,3.175238,3.201141,1003.0,0.103171
3,Grumpier Old Men (1995),478,3.073529,2.994152,3.016736,1152.0,0.079377
5,Father of the Bride Part II (1995),296,3.212963,2.888298,3.006757,1054.0,0.324665
6,Heat (1995),940,3.682171,3.909988,3.878723,591.0,0.227817
...,...,...,...,...,...,...,...
3930,"Creature From the Black Lagoon, The (1954)",223,3.121951,3.285714,3.255605,1121.0,0.163763
3932,"Invisible Man, The (1933)",232,3.681818,3.765957,3.750000,592.0,0.084139
3948,Meet the Parents (2000),862,3.619048,3.641838,3.635731,664.0,0.022791


In [165]:
# 10. 평점의 표준편차가 큰 영화
movie_data = data_result.groupby(['movieId','title'])
ex10_frm= movie_data['rating'].std()
ex10_frm = pd.DataFrame(ex10_frm[up_200])
ex10_frm.columns=['std']
ex10_frm_result = ex10_frm.sort_values(by='std',ascending=False)
ex10_frm_result

,,std
movieId,title,
1924,Plan 9 from Outer Space (1958),1.455998
2459,"Texas Chainsaw Massacre, The (1974)",1.332448
231,Dumb & Dumber (1994),1.321333
2710,"Blair Witch Project, The (1999)",1.316368
288,Natural Born Killers (1994),1.307198
...,...,...
1148,"Wrong Trousers, The (1993)",0.708666
318,"Shawshank Redemption, The (1994)",0.700443
1262,"Great Escape, The (1963)",0.692585


In [186]:
# Comedy영화 중 가장 평점이  낮은 영화의 제목
comedy_frm = data_result[data_result['genres'].str.contains("Comedy")]
comedy_frm = comedy_frm[['title','genres','rating']]
comedy_frm.groupby('title').agg(np.mean).sort_values(by='rating',ascending=True)

,rating
title,
Sleepover (1995),1.000000
Spring Fever USA (a.k.a. Lauderdale) (1989),1.000000
Nueba Yol (1995),1.000000
Santa with Muscles (1996),1.000000
Waltzes from Vienna (1933),1.000000
...,...
"Wrong Trousers, The (1993)",4.507937
"Close Shave, A (1995)",4.520548
Follow the Bitch (1998),5.000000


In [187]:
# 평균 평점이 가장 높은 영화의 제목(동률이 있을 경우 모두 출력)
max_rating_mean = ex7_frm['All']==np.max(ex7_frm['All'])
ex7_frm[max_rating_mean]

,,rating_cnt,F,M,All,female_rank,gender_diff
movieId,title,,,,,,
2019,Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954),628,4.481132,4.576628,4.56051,14.0,0.095496
